# **Research Paper Recommendation System**

TEAM 01

Zeba Fathima - 1RVU22BSC031

Jyosthna Chidarapadu - 1RVU22BSC021

In [ ]:
!pip install feedparser

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 709.3 kB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=5522a962a049e7dc60b284f30af1790f3c929fb0da98a58a3b90c7e2d34d250a
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [ ]:
import feedparser
from urllib.parse import quote
from sentence_transformers import SentenceTransformer, util
from datetime import datetime
import numpy as np
import re
# from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import spacy

In [ ]:
# Initialize NLP components
print("Loading NLP models...")
model = SentenceTransformer('all-MiniLM-L6-v2') #embedings semantic similarity
nlp = spacy.load("en_core_web_sm") #keyword extraction


Loading NLP models...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def clean_text(text):
    """Basic text preprocessing"""
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special chars
    return text

In [ ]:
### arXiv Search Functions ###
def search_arxiv(query, max_results=10, sort_by="relevance"):
    """Search arXiv API"""
    base_url = 'http://export.arxiv.org/api/query?' ## Base URL of the arXiv API to send queries to
    encoded_query = quote(query)

    query_url = f"search_query=all:{encoded_query}&start=0&max_results={max_results}"

    if sort_by in ['submittedDate', 'relevance']:
        query_url += f"&sortBy={sort_by}&sortOrder=descending"

    ## Send the full API request and parse the response
    parsed = feedparser.parse(base_url + query_url)

    papers = []
    for entry in parsed.entries: ## Loop through each paper entry in the parsed API response
        title = entry.title.replace('\n', ' ').strip()
        authors = [author.name for author in entry.authors]
        link = entry.link
        abstract = entry.summary.replace('\n', ' ').strip() if 'summary' in entry else ""
        published = entry.published
        year = published.split('-')[0] if published else "Unknown"
        submission_date = entry.get('arxiv_updated', published)

        try:
            date_obj = datetime.strptime(submission_date, '%Y-%m-%dT%H:%M:%SZ')
        except:
            # date_obj = datetime.now()
            date_obj = None

        papers.append({
            'title': title,
            'authors': authors,
            'abstract': abstract,
            'year': year,
            'link': link,
            'date_obj': date_obj,
            'date_str': submission_date
        })

    return papers

In [ ]:
### Key Phrase Extraction ###
def extract_keywords(text):
    """Extract key phrases using noun phrase chunking"""
    try:
        # SpaCy noun phrase extraction
        doc = nlp(text)
        noun_phrases = [chunk.text for chunk in doc.noun_chunks]

        # Get most common noun phrases
        phrase_counts = Counter(noun_phrases)
        top_phrases = [phrase for phrase, count in phrase_counts.most_common(5)]
        return top_phrases

    except Exception as e:
        print(f"Keyword extraction error: {str(e)}")
        return []

In [ ]:
### Semantic Search & Ranking ###
def hybrid_sort(papers, query_embedding, recency_weight=0.3):
    """Sorting combining semantic and temporal features"""
    if not papers:
        return []

    current_date = datetime.now()

    # Calculate semantic similarity
    paper_titles = [paper['title'] for paper in papers]
    title_embeddings = model.encode(paper_titles, convert_to_tensor=True)
    title_similarity = util.cos_sim(query_embedding, title_embeddings)[0].cpu().numpy()

    relevance_scores = title_similarity

    # Recency scoring
    dates = [paper['date_obj'] for paper in papers]
    date_diffs = [(current_date - date).days for date in dates]
    max_diff = max(date_diffs) if date_diffs else 1
    recency_scores = 1 - np.array(date_diffs) / max_diff

    # Final combined scores
    combined_scores = (1 - recency_weight) * relevance_scores + recency_weight * recency_scores

    # Sort results
    sorted_indices = np.argsort(combined_scores)[::-1]
    sorted_papers = [papers[i] for i in sorted_indices]

    for i, paper in enumerate(sorted_papers):
        paper['relevance_score'] = float(relevance_scores[sorted_indices[i]])
        paper['recency_score'] = float(recency_scores[sorted_indices[i]])
        paper['combined_score'] = float(combined_scores[sorted_indices[i]])

        # Add keyword extraction
        if paper.get('abstract'):
            paper['keywords'] = extract_keywords(paper['abstract'])

    return sorted_papers

In [ ]:
def format_citation(title, authors, year, link, style='APA'):
    """Format citation"""
    if style == 'APA':
        authors_str = ', '.join(authors[:2]) + (' et al.' if len(authors) > 2 else '')
        return f"{authors_str} ({year}). {title}. {link}"
    else:
        return f"{title} ({year}) - {link}"


In [ ]:
### Recommendation System Core ###
def real_time_recommendations_with_citations(query, max_results=5, recency_weight=0.3):
    """Get recommendations with keyword extraction"""
    # First get papers sorted by relevance from arXiv
    relevant_papers = search_arxiv(query, max_results=20, sort_by='relevance')

    # Then get recent papers from arXiv
    recent_papers = search_arxiv(query, max_results=20, sort_by='submittedDate')

    # Combine and deduplicate
    all_papers = relevant_papers + recent_papers
    unique_papers = {paper['title']: paper for paper in all_papers}.values()

    if not unique_papers:
        return []

    clean_query = clean_text(query)
    query_embedding = model.encode(clean_query, convert_to_tensor=True)

    # Apply hybrid sorting with keyword extraction
    sorted_papers = hybrid_sort(list(unique_papers), query_embedding, recency_weight)

    # Prepare final results with keywords
    results = []
    for paper in sorted_papers[:max_results]:
        results.append({
            'title': paper['title'],
            'authors': paper['authors'],
            'year': paper['year'],
            'link': paper['link'],
            'date': paper['date_str'],
            'abstract': paper.get('abstract', ''),
            'relevance': paper['relevance_score'],
            'recency': paper['recency_score'],
            'combined_score': paper['combined_score'],
            'keywords': paper.get('keywords', []),
            'citation': format_citation(
                paper['title'],
                paper['authors'],
                paper['year'],
                paper['link']
            )
        })

    return results

In [ ]:
### User Interface Functions ###
def get_paper_details(paper):
    """Display paper information with keywords"""
    print("\n=== Paper Details ===")
    print(f"\nTitle: {paper['title']}")
    print(f"\nAuthors: {', '.join(paper['authors'])}")
    print(f"\nPublished: {paper['date'].split('T')[0]}")

    if paper.get('abstract'):
        print(f"\nAbstract:\n{paper['abstract']}")
        if paper.get('keywords'):
            print(f"\nKey Phrases: {', '.join(paper['keywords'])}")

    print(f"\nCitation:\n{paper['citation']}")
    print(f"\nLink: {paper['link']}")


In [ ]:
def user_interface():
    """Main interactive interface"""
    print("\nResearch Paper Recommendation System")
    print("===================================")
    print("Features:")
    print("- Semantic search with ranking")
    print("- Key phrase extraction from abstracts\n")

    while True:
        print("\nEnter a research topic or paper title (or 'quit' to exit):")
        user_input = input("> ").strip()

        if user_input.lower() in ['quit', 'exit']:
            print("Goodbye!")
            break

        if len(user_input) < 3:
            print("Please enter a more detailed query (at least 3 characters)")
            continue

        print("\nSearching arXiv and analyzing papers...")
        try:
            recommendations = real_time_recommendations_with_citations(user_input)

            if not recommendations:
                print("No papers found. Try a different search term.")
                continue

            # Display top recommendations
            print("\nTop recommendations:")
            for i, paper in enumerate(recommendations, 1):
                print(f"\n{i}. {paper['title']}")
                print(f"   Score: {paper['combined_score']:.3f} (Relevance: {paper['relevance']:.3f}, Recency: {paper['recency']:.3f})")

            # Paper exploration menu
            while True:
                print("\nOptions:")
                print("- Enter a paper number (1-5) for detailed analysis")
                print("- Enter 'all' to see all details")
                print("- Enter 'back' to search again")
                print("- Enter 'quit' to exit")
                choice = input("> ").strip().lower()

                if choice == 'quit':
                    return
                elif choice == 'back':
                    break
                elif choice == 'all':
                    for paper in recommendations:
                        get_paper_details(paper)
                elif choice.isdigit() and 1 <= int(choice) <= len(recommendations):
                    selected_paper = recommendations[int(choice)-1]
                    get_paper_details(selected_paper)
                else:
                    print("Invalid input. Please try again.")

        except Exception as e:
            print(f"\nError: {str(e)}")
            print("Please try again or check your internet connection.")

In [ ]:
if __name__ == "__main__":
    user_interface()



Research Paper Recommendation System
Features:
- Semantic search with ranking
- Key phrase extraction from abstracts


Enter a research topic or paper title (or 'quit' to exit):
> ML in health care

Searching arXiv and analyzing papers...

Top recommendations:

1. Machine Learning for Everyone: Simplifying Healthcare Analytics with   BigQuery ML
   Score: 0.646 (Relevance: 0.506, Recency: 0.972)

2. Uncertainty Quantification for Machine Learning in Healthcare: A Survey
   Score: 0.607 (Relevance: 0.441, Recency: 0.995)

3. Development and Validation of ML-DQA -- a Machine Learning Data Quality   Assurance Framework for Healthcare
   Score: 0.559 (Relevance: 0.496, Recency: 0.707)

4. The Transition from Centralized Machine Learning to Federated Learning   for Mental Health in Education: A Survey of Current Methods and Future   Directions
   Score: 0.481 (Relevance: 0.273, Recency: 0.966)

5. Ethical Machine Learning in Health Care
   Score: 0.479 (Relevance: 0.465, Recency: 0.512)

O

## **Evaluation**

In [ ]:
queries_with_results = [
    ("machine learning in heart-attack detection", real_time_recommendations_with_citations("machine learning in heart-attack detection")),
    ("deep learning in plant disease detection", real_time_recommendations_with_citations("deep learning in plant disease detection")),
    ("generative AI impact on creative jobs", real_time_recommendations_with_citations("generative AI impact on creative jobs")),
    ("algorithmic bias in hiring systems", real_time_recommendations_with_citations("algorithmic bias in hiring systems")),
    ("generative AI impact on creative jobs", real_time_recommendations_with_citations("generative AI impact on creative jobs")),
     ("transformer architecture in deep learning", real_time_recommendations_with_citations("transformer architecture in deep learning")),
     ("GNN for social network analysis", real_time_recommendations_with_citations("GNN for social network analysis")),
     ("5G network slicing optimization", real_time_recommendations_with_citations("5G network slicing optimization")),
     ("NFT art market analysis", real_time_recommendations_with_citations("NFT art market analysis")),
     ("liquid neural networks for robotics control", real_time_recommendations_with_citations("liquid neural networks for robotics control")),
     ("battery recycling lithium recovery methods", real_time_recommendations_with_citations("battery recycling lithium recovery methods")),
     ("photoacoustic imaging tumor margin detection", real_time_recommendations_with_citations("photoacoustic imaging tumor margin detection")),
     ("quantum sensing for neurodegenerative diseases", real_time_recommendations_with_citations("quantum sensing for neurodegenerative diseases")),
     ("metamaterials for gravitational wave detection", real_time_recommendations_with_citations("metamaterials for gravitational wave detection")),
     ("AI watermarking for generated content", real_time_recommendations_with_citations("AI watermarking for generated content")),
     ("biodegradable sensors for agriculture", real_time_recommendations_with_citations("biodegradable sensors for agriculture")),
     ("organoid intelligence computing with neurons", real_time_recommendations_with_citations("organoid intelligence computing with neurons")),
     ("agrivoltaics system optimization techniques", real_time_recommendations_with_citations("agrivoltaics system optimization techniques")),
     ("multimodal foundation models for healthcare", real_time_recommendations_with_citations("multimodal foundation models for healthcare")),
     ("photonic neuromorphic computing for edge AI", real_time_recommendations_with_citations("photonic neuromorphic computing for edge AI")),
     ("nanopore-based early cancer detection using cell-free DNA", real_time_recommendations_with_citations("nanopore-based early cancer detection using cell-free DNA")),
     ("machine learning for wildfire spread prediction with satellite data", real_time_recommendations_with_citations("machine learning for wildfire spread prediction with satellite data")),
     ("error-corrected logical qubits using bosonic codes", real_time_recommendations_with_citations("error-corrected logical qubits using bosonic codes")),
     ("closed-loop deep brain stimulation for obsessive-compulsive disorder", real_time_recommendations_with_citations("closed-loop deep brain stimulation for obsessive-compulsive disorder")),
     ("self-healing hydrogels for soft robotics with ionic conductivity", real_time_recommendations_with_citations("self-healing hydrogels for soft robotics with ionic conductivity"))]


In [ ]:
### Simplified Evaluation Metrics ###
def evaluate_recommendations(queries_with_results):
    """
    Evaluate recommendation quality for multiple queries

    Args:
        queries_with_results: List of tuples (query, recommendations)

    Returns:
        Dictionary containing aggregate metrics
    """
    if not queries_with_results:
        return {}

    # Initialize metrics
    metrics = {
        'total_queries': len(queries_with_results),
        'avg_papers_per_query': 0,
        'avg_relevance': 0,
        'avg_recency': 0,
        'avg_combined_score': 0,
        'success_rate': 0
    }

    successful_queries = 0

    for query, recommendations in queries_with_results:
        if not recommendations:
            continue

        successful_queries += 1
        metrics['avg_papers_per_query'] += len(recommendations)
        metrics['avg_relevance'] += np.mean([p['relevance'] for p in recommendations])
        metrics['avg_recency'] += np.mean([p['recency'] for p in recommendations])
        metrics['avg_combined_score'] += np.mean([p['combined_score'] for p in recommendations])

    if successful_queries > 0:
        metrics['avg_papers_per_query'] /= successful_queries
        metrics['avg_relevance'] /= successful_queries
        metrics['avg_recency'] /= successful_queries
        metrics['avg_combined_score'] /= successful_queries
        metrics['success_rate'] = successful_queries / metrics['total_queries']

    return metrics


In [ ]:
metrics = evaluate_recommendations(queries_with_results)

In [ ]:
def print_evaluation_report(metrics):
    """Print evaluation results in a simple format"""
    print("\n=== Evaluation Report ===")
    print(f"Total queries evaluated: {metrics['total_queries']}")
    print(f"Success rate: {metrics['success_rate']:.1%}")
    print(f"Average papers per query: {metrics['avg_papers_per_query']:.1f}")
    print(f"Average relevance score: {metrics['avg_relevance']:.3f}")
    print(f"Average recency score: {metrics['avg_recency']:.3f}")
    print(f"Average combined score: {metrics['avg_combined_score']:.3f}")

In [ ]:
print_evaluation_report(metrics)


=== Evaluation Report ===
Total queries evaluated: 25
Success rate: 100.0%
Average papers per query: 5.0
Average relevance score: 0.615
Average recency score: 0.847
Average combined score: 0.685
